In [8]:
# !python -m pip install mlagents==0.28.0
# !python -m pip install gym
# !cd "/Users/aditya/Documents/GitHub/game_creation_research/ml-agents/gym-unity" && pip3 install -e .

import mlagents
import numpy as np
import json
from mlagents_envs.environment import UnityEnvironment
from collections import namedtuple, defaultdict
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from gym_unity.envs import UnityToGymWrapper


In [11]:
from typing import List

class Obs():
    def __init__(self, raw_obs):
        """
        Converts Unity Agent outputted Vector Observation to 
        named format
        """
        self.raw_obs = raw_obs
        self.objectOrder = ["corner", "bucket", "triangle", "gear", "crate"]
        self.objPos = {}
        self.objPos[self.objectOrder[0]] = self.Vector2(raw_obs[0], raw_obs[1])
        self.objPos[self.objectOrder[1]] = self.Vector2(raw_obs[2], raw_obs[3])
        self.objPos[self.objectOrder[2]] = self.Vector2(raw_obs[4], raw_obs[5])
        self.objPos[self.objectOrder[3]] = self.Vector2(raw_obs[6], raw_obs[7])
        self.objPos[self.objectOrder[4]] = self.Vector2(raw_obs[8], raw_obs[9])
        self.ballPos = self.Vector2(raw_obs[10], raw_obs[11])
        self.ballVel = self.Vector2(raw_obs[12], raw_obs[13])
        self.reset = bool(raw_obs[14])

    def show(self) -> None:
        """
        Pretty Print Observation
        """
        for each_obj in self.objPos:
            print(f"{each_obj}: {self.objPos[each_obj]}")
        print(f"Ball Position: {self.ballPos}")
        print(f"Ball Velocity: {self.ballVel}")
        print(f"In Reset?: {self.reset}")
    
    def toArray(self):
        return self.raw_obs

class PlaceAndShootGym(UnityToGymWrapper):
    def __init__(self, gym_env, reward_fn):
        self.gym_env = gym_env
        self.reward_fn = reward_fn
        # unsure if this is always true
        self.Vector2 = namedtuple('Vector2', 'x y')
        self.velTresh = 0.001

    def step(self, action):
        """
        Step is defined as doing something ball has stopped
        """
        obsVec = []
        # first step
        raw_obs, _reward, _done, info = self.gym_env.step(action)
        obsVec.append(Obs(raw_obs))
        # continued steps
        while (any([abs(f)>self.velTresh for f in obsVec[-1].ballVel])):
            raw_obs, _reward, done, info = self.gym_env.step(action)
            obsVec.append(Obs(raw_obs))
        reward = self.getRewards(obsVec)
        return (obsVec[-1].toArray(), reward, done, info)
    
    def getRewards(self, obsVec: List[Obs]) -> float:
        return float(self.reward_fn(obsVec))

    def close(self):
        self.gym_env.close()


In [ ]:
def endsInBucket(obsVec: List[Obs]) -> bool:
    """
    Custom Reward Fn:
    Is that ball in bucket at the end or no?
    """
    bucket_pos = obsVec[-1].objPos["bucket"]
    obsVec[-1].ballPos
    

In [5]:
SERVER_BUILD = "../Builds/MLAgent_View_21April22_server.app"
GRAPHICAL_BUILD = "../Builds/MLAgent_View_21April22.app"
GYM_BUILD = "../Builds/Gym_View_22April22.app"

# channel = EngineConfigurationChannel()
# channel.set_configuration_parameters(time_scale = 1.0, quality_level=5)
# unity_env = UnityEnvironment(file_name=GYM_BUILD, seed=1, side_channels=[channel], worker_id=1)

unity_env = UnityEnvironment()

# Start interacting with the environment.
unity_env.reset()
gym_env = UnityToGymWrapper(unity_env, allow_multiple_obs=False)
env = PlaceAndShootGym(gym_env)


[INFO] Listening on port 5004. Start training by pressing the Play button in the Unity Editor.
[INFO] Connected to Unity environment with package version 2.2.1-exp.1 and communication version 1.5.0
[INFO] Connected new brain: PlaceAndShoot?team=0
[WARNING] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.


/opt/anaconda3/envs/introml_conda/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [7]:
action = [-1.4, -1.5, 0, 0, 0, 0]
env.step(action)

Vector2(x=9.352689, y=13.661685)
Vector2(x=10.8729105, y=-5.8132052)
Vector2(x=-5.2235017, y=-10.170062)
Vector2(x=-5.018889, y=-13.614484)
Vector2(x=-6.140332, y=3.2639103)
Vector2(x=-5.899805, y=-0.70673394)
Vector2(x=2.83435, y=-1.9715983)
Vector2(x=2.7233236, y=-5.7371573)
Vector2(x=1.4265316, y=0.3329459)
Vector2(x=1.3885376, y=1.4677314)
Vector2(x=1.3418412, y=0.04493957)
Vector2(x=1.3008544, y=0.0)
Vector2(x=1.2583771, y=0.0)
Vector2(x=1.2172506, y=0.0)
Vector2(x=1.1774685, y=0.0)
Vector2(x=1.1389862, y=0.0)
Vector2(x=1.101762, y=0.0)
Vector2(x=1.0657543, y=0.0)
Vector2(x=1.0309235, y=0.0)
Vector2(x=0.997231, y=0.0)
Vector2(x=-0.0012699474, y=0.0)
Vector2(x=-0.0012283695, y=0.0)
Vector2(x=-0.0011881532, y=0.0)
Vector2(x=-0.0011492533, y=0.0)
Vector2(x=-0.0011115279, y=0.0)
Vector2(x=-0.0010751366, y=0.0)
Vector2(x=-0.0010399367, y=0.0)
Vector2(x=-0.0010058896, y=0.0)
Vector2(x=-0.0009729569, y=0.0)


{'objPos': {'corner': Vector2(x=5.34, y=-2.65),
  'bucket': Vector2(x=5.34, y=1.85),
  'triangle': Vector2(x=5.34, y=-1.25),
  'gear': Vector2(x=5.34, y=0.24),
  'crate': Vector2(x=5.34, y=3.26)},
 'ballPos': Vector2(x=2.683598, y=-4.603575),
 'ballVel': Vector2(x=-0.0009729569, y=0.0),
 'reset': False}

In [10]:
env.close()